In [1]:
import pandas as pd
import numpy as np
from typing import *

In [2]:
true_df = pd.read_csv("./Joseph_Dataset/cropped_rcnn_annotations.csv")
false_df = pd.read_csv("./Joseph_Dataset/incorrect_rcnn_preds.csv")
df = pd.concat([true_df, false_df], sort=False)
labels = np.concatenate([np.ones([len(true_df)]), np.zeros([len(false_df)])])

In [3]:
from keras.layers import Dense, BatchNormalization
from keras.layers.recurrent import GRU
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
input_data = np.stack((
    df['xmin'], df['xmax'], df['ymin'], df['ymax'],
    df['xmax'] - df['xmin'],
    df['ymax'] - df['ymin'],
    df['confidence']
), axis=-1)

np.random.seed(1234)
indices = np.random.randint(len(df), size=len(df))
input_data = input_data[indices]
labels = labels[indices]

In [67]:
LOSS_BIAS = 0.9
from keras import backend as K

def weighted_log_loss(yt, yp) -> Any:
    """
    Binary crossentropy with a bias towards false negatives
    yt: Target
    yp: Prediction
    """
    from keras import backend as K

    pos_loss = -(0 + yt) * K.log(0 + yp + K.epsilon())
    neg_loss = -(1 - yt) * K.log(1 - yp + K.epsilon())

    return LOSS_BIAS * K.mean(neg_loss) + (1. - LOSS_BIAS) * K.mean(pos_loss)


def false_pos(yt, yp) -> Any:
    from keras import backend as K
    return K.sum(K.cast(yp * (1 - yt) > 0.5, 'float')) / K.maximum(1.0, K.sum(1 - yt))


def false_neg(yt, yp) -> Any:
    from keras import backend as K
    return K.sum(K.cast((1 - yp) * (0 + yt) > 0.5, 'float')) / K.maximum(1.0, K.sum(0 + yt))

def threshold_binary_accuracy(y_true, y_pred):
    threshold = 0.5
    return K.mean(K.equal(y_true, K.cast(y_pred>threshold, 'float')))

In [48]:
model = Sequential()
model.add(Dense(40, input_shape=(input_data.shape[1],), activation='sigmoid'))
model.add(Dense(40, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'mse', metrics=['acc', threshold_binary_accuracy, false_pos, false_neg])

In [49]:
train_ratio = 0.8
train_split = int(len(input_data) * train_ratio)
train_in = input_data[:train_split]
train_out = labels[:train_split]
test_in = input_data[train_split:]
test_out = labels[train_split:]

In [50]:
model.fit(train_in, train_out, 64, 100, validation_data=(test_in, test_out))

Train on 50073 samples, validate on 12519 samples
Epoch 1/100
37632/50073 [=====================>........] - ETA: 0s - loss: 0.2000 - acc: 0.6951 - threshold_binary_accuracy: 0.5849 - false_pos: 0.9253 - false_neg: 0.0211

KeyboardInterrupt: 

In [68]:
model2 = Sequential()
model2.add(BatchNormalization(input_shape=(input_data.shape[1],)))
model2.add(Dense(40, activation='relu'))
model2.add(Dense(40, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
class_weight = {0: 0.3,
                1: 0.7,
                }
model2.compile('rmsprop', 'mse',  metrics=['acc', threshold_binary_accuracy, false_pos, false_neg])

In [ ]:
model2.fit(train_in, train_out, 64, 100, class_weight = class_weight, validation_data=(test_in, test_out))

Train on 50073 samples, validate on 12519 samples
Epoch 1/100
50073/50073 [==============================] - 4s 82us/step - loss: 0.0620 - acc: 0.7386 - threshold_binary_accuracy: 0.7386 - false_pos: 0.7399 - false_neg: 0.0377 - val_loss: 0.1482 - val_acc: 0.7785 - val_threshold_binary_accuracy: 0.7785 - val_false_pos: 0.5777 - val_false_neg: 0.0592
Epoch 2/100
50073/50073 [==============================] - 3s 63us/step - loss: 0.0581 - acc: 0.7548 - threshold_binary_accuracy: 0.7548 - false_pos: 0.6838 - false_neg: 0.0397 - val_loss: 0.1609 - val_acc: 0.7380 - val_threshold_binary_accuracy: 0.7380 - val_false_pos: 0.8059 - val_false_neg: 0.0132
Epoch 3/100
50073/50073 [==============================] - 3s 64us/step - loss: 0.0576 - acc: 0.7559 - threshold_binary_accuracy: 0.7559 - false_pos: 0.6803 - false_neg: 0.0399 - val_loss: 0.1517 - val_acc: 0.7667 - val_threshold_binary_accuracy: 0.7667 - val_false_pos: 0.6401 - val_false_neg: 0.0479
Epoch 4/100
50073/50073 [===================

Epoch 28/100
50073/50073 [==============================] - 3s 64us/step - loss: 0.0562 - acc: 0.7643 - threshold_binary_accuracy: 0.7643 - false_pos: 0.6522 - false_neg: 0.0411 - val_loss: 0.1483 - val_acc: 0.7703 - val_threshold_binary_accuracy: 0.7703 - val_false_pos: 0.6496 - val_false_neg: 0.0380
Epoch 29/100
50073/50073 [==============================] - 3s 63us/step - loss: 0.0563 - acc: 0.7638 - threshold_binary_accuracy: 0.7638 - false_pos: 0.6516 - false_neg: 0.0411 - val_loss: 0.1539 - val_acc: 0.7596 - val_threshold_binary_accuracy: 0.7596 - val_false_pos: 0.7061 - val_false_neg: 0.0275
Epoch 30/100
50073/50073 [==============================] - 3s 64us/step - loss: 0.0562 - acc: 0.7645 - threshold_binary_accuracy: 0.7645 - false_pos: 0.6526 - false_neg: 0.0406 - val_loss: 0.1482 - val_acc: 0.7664 - val_threshold_binary_accuracy: 0.7664 - val_false_pos: 0.6712 - val_false_neg: 0.0336
Epoch 31/100
50073/50073 [==============================] - 3s 63us/step - loss: 0.0563 - a

In [15]:
model2.save_weights("model.h5")